<a href="https://colab.research.google.com/github/Prasanna28Devadiga/Portfolio-Allocation-DRL/blob/master/Custom_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip3 install gym
import numpy as np
import gym
import random
import json
from gym import spaces
import pandas as pd
!pip3 install stable_baselines3


     |████████████████████████████████| 153kB 5.4MB/s 


In [20]:
class StockTradingEnvironment(gym.Env):
  """A stock trading environment for OpenAI gym"""
  metadata = {'render.modes': ['human']}
  def __init__(self, df):   # we'll be passing a pandas df as input
    super(StockTradingEnv, self).__init__()
    self.df = df
    self.reward_range = (0, MAX_ACCOUNT_BALANCE)   
    # Actions of the format Buy x%, Sell x%, Hold   -> buy- [0,1), sell-[1,2) hold - [2,3)
    self.action_space = spaces.Box(
      low=np.array([0, 0]), high=np.array([3, 1]), dtype=np.float16) 
    # Prices contains the OHCL values for the last five prices
    self.observation_space = spaces.Box(
      low=0, high=1, shape=(6, 6), dtype=np.float16)

def _next_observation(self):
        # Get the stock data points for the last 5 days and scale to between 0-1
        frame = np.array([
            self.df.loc[self.current_step: self.current_step +
                        5, 'Open'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'High'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'Low'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'Close'].values / MAX_SHARE_PRICE,
            self.df.loc[self.current_step: self.current_step +
                        5, 'Volume'].values / MAX_NUM_SHARES,
        ])

        # Append additional data and scale each value to between 0-1
        obs = np.append(frame, [[
            self.balance / MAX_ACCOUNT_BALANCE,
            self.max_net_worth / MAX_ACCOUNT_BALANCE,
            self.shares_held / MAX_NUM_SHARES,
            self.cost_basis / MAX_SHARE_PRICE,
            self.total_shares_sold / MAX_NUM_SHARES,
            self.total_sales_value / (MAX_NUM_SHARES * MAX_SHARE_PRICE),
            self.change,
            self.rsi,
            self.mfi, #normalise mfi
            self.SMA50_Close,  #normalise 
            self.SMA100_Close, #normalise
            self.SMA200_Close, #normalise
            self.signal_macd_Close,
            self.macd_Close

        ]], axis=0)

        return obs

In [9]:
def step(self, action):
  # Execute one time step within the environment
  self._take_action(action)
  self.current_step += 1
  if self.current_step > len(self.df.loc[:, 'Open'].values) - 6:
    self.current_step = 0
  delay_modifier = (self.current_step / MAX_STEPS)
  
  reward = self.balance * delay_modifier
  done = self.net_worth <= 0
  obs = self._next_observation()
  return obs, reward, done, {}

In [10]:
def _take_action(self, action):
  # Set the current price to a random price within the time step
  current_price = random.uniform(
    self.df.loc[self.current_step, "Open"],
    self.df.loc[self.current_step, "Close"])
  action_type = action[0]
  amount = action[1]
  if action_type < 1:
    # Buy amount % of balance in shares
    total_possible = self.balance / current_price
    shares_bought = total_possible * amount
    prev_cost = self.cost_basis * self.shares_held
    additional_cost = shares_bought * current_price
    self.balance -= additional_cost
    self.cost_basis = (prev_cost + additional_cost) /(self.shares_held + shares_bought)
    self.shares_held += shares_bought
  elif actionType < 2:
    # Sell amount % of shares held
    shares_sold = self.shares_held * amount
    self.balance += shares_sold * current_price
    self.shares_held -= shares_sold
    self.total_shares_sold += shares_sold
    self.total_sales_value += shares_sold * current_price
  self.netWorth = self.balance + self.shares_held * current_price
  if self.net_worth > self.max_net_worth:
    self.max_net_worth = net_worth
  if self.shares_held == 0:
    self.cost_basis = 0

In [15]:
def render(self, mode='human', close=False):
  # Render the environment to the screen
  profit = self.net_worth - INITIAL_ACCOUNT_BALANCE
  print(f'Step: {self.current_step}')
  print(f'Balance: {self.balance}')
  print(f'Shares held: {self.shares_held}')
  print(f'Total sold: {self.total_shares_sold}')
  print(f'Avg cost for held shares: {self.cost_basis}')
  print(f'Total sales value: {self.total_sales_value}')
  print(f'Net worth: {self.net_worth}')
  print(f'Max net worth: {self.max_net_worth}')
  print(f'Profit: {profit}')

In [18]:
import gym
import json
import datetime as dt
from stable_baselines.common.policies import MlpPolicy #gotta replace with stable baselines 3 and test it out
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from env.StockTradingEnv import StockTradingEnv
import pandas as pd
df = pd.read_csv('./data/nasdaq100.csv')
df = df.sort_values('Date')
# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: StockTradingEnv(df)])
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000)
obs = env.reset()
for i in range(2000):
  action, _states = model.predict(obs)
  obs, rewards, done, info = env.step(action)
  env.render()

ModuleNotFoundError: ignored